In [4]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [5]:
#!pip install catboost

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
myfolder = "./"

# **Columns' names**

In [8]:
#Columns' names
'''
1)  unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  21
'''
unitNames = ['UnitNumber']
timeCycles = ["TimeInCycles"]
operSets = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes = ["SensorMes"+str(j) for j in range(1, 22)] # de 1 à 21
columnsNames = unitNames + timeCycles + operSets +sensorMes

# **Datasets loading**

In [9]:
def data_loading(x):
  train_path = myfolder + "train_"+ x +".txt"
  test_path = myfolder + "test_"+ x +".txt"
  rul_path = myfolder + "RUL_"+ x +".txt"
  train = pd.read_csv(train_path, delim_whitespace=True, names=columnsNames)
  test = pd.read_csv(test_path, delim_whitespace=True, names=columnsNames)
  rul = pd.read_csv(rul_path, delim_whitespace=True, names=["RUL_FD"])
  return train, test, rul

train_fd001, test_fd001, rul_fd001 = data_loading("FD001")
train_fd002, test_fd002, rul_fd002 = data_loading("FD002")
train_fd003, test_fd003, rul_fd003 = data_loading("FD003")
train_fd004, test_fd004, rul_fd004 = data_loading("FD004")

In [10]:
# Forcer l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213


# **RUL column generation for train and test set**

In [11]:
def rul_train_generation(x):
  rul = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  rul.columns = ['UnitNumber', 'max']
  x = x.merge(rul, on=['UnitNumber'], how='left')
  x['RUL'] = x['max'] - x['TimeInCycles']
  x.drop('max', axis=1, inplace=True)
  return x

train_fd001 = rul_train_generation(train_fd001)
train_fd002 = rul_train_generation(train_fd002)
train_fd003 = rul_train_generation(train_fd003)
train_fd004 = rul_train_generation(train_fd004)

In [12]:
train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213,318


In [13]:
def rul_test_generation(x, rul):

  rul["UnitNumber"] = rul.index + 1 # +1 pour que UnitNumber demarre de 1 au lieu de 0, car il s'agit du numero des moteur

  x = x.merge(rul, on=['UnitNumber'], how='left')

  max_cycle = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  max_cycle.columns = ['UnitNumber', 'max']
  x = x.merge(max_cycle, on=['UnitNumber'], how='left')
  x['RUL'] = x['RUL_FD'] + x['max'] - x['TimeInCycles']
  x.drop(['max', 'RUL_FD'], axis=1, inplace=True)

  return x

test_fd001 = rul_test_generation(test_fd001, rul_fd001)
test_fd002 = rul_test_generation(test_fd002, rul_fd002)
test_fd003 = rul_test_generation(test_fd003, rul_fd003)
test_fd004 = rul_test_generation(test_fd004, rul_fd004)

In [14]:
test_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,20.0072,0.700,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,332.52,2323.67,8704.98,1.07,43.83,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,251
1,1,2,24.9984,0.620,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,174.46,1915.21,7999.94,0.93,36.11,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,250
2,1,3,42.0000,0.842,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,137.34,2211.93,8305.38,1.01,41.52,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,249


# **RANDOM SAMPLE SELECTION**

In [15]:
'''element_counts = test_fd002[test_fd002['UnitNumber']==1]
len(element_counts)'''

"element_counts = test_fd002[test_fd002['UnitNumber']==1]\nlen(element_counts)"

In [16]:
import random

def selection_aleatoire(df, sample_size, rand_state):
    unique_values = df["UnitNumber"].unique()
    selected_rows = []
    for value in unique_values:
        rows = df[df["UnitNumber"] == value]
        if len(rows) < sample_size : # si la taille de l'echantillon donnée est superieur au nombre total de ligne pour un moteur, reinitialiser la valeur
            sample_size = len(rows)
        random_sample = rows.sample(n=sample_size, random_state = rand_state)  # Sélectionne 50 lignes aléatoires
        selected_rows.append(random_sample)
    result = pd.concat(selected_rows)
    return result

# **Data normalization**

In [17]:
def normalised_df(train, test):

  from sklearn.preprocessing import MinMaxScaler

  # Instancier l'objet MinMaxScaler pour normaliser les données
  scaler = MinMaxScaler()

  # Normaliser train
  train_scaled = scaler.fit_transform(train)
  train_df = pd.DataFrame(train_scaled)

  # Normaliser test
  test_scaled = scaler.fit_transform(test)
  test_df = pd.DataFrame(test_scaled)


  train_df.columns = train.columns
  test_df.columns = test.columns
    
  return train_df, test_df

# **Data splitting**

In [18]:
def data_split(train, test):

  # data split
  X_train = train.drop('RUL', axis=1)
  Y_train = train['RUL']
  X_test = test.drop('RUL', axis=1)
  Y_test = test['RUL']
    
  return X_train, Y_train, X_test, Y_test


# **The KNN Regressor model**

In [19]:
import time

def my_knn_regressor(x, y):

    # Time tracking, Operation time (min)
    t = time.process_time()
    
    t = time.process_time()

    mse_test_list = []
    rmse_test_list = []
    mae_test_list = []
    mape_test_list = []
    
    for j in range(1, 11):
        
        normalized_train_df, normalized_test_df =  normalised_df(x, y)
        
        train_selected = selection_aleatoire(normalized_train_df, 50, j)
        test_selected = selection_aleatoire(normalized_test_df, 25, j)


        X_train, Y_train, X_test, Y_test = data_split(train_selected, test_selected)

        '''print("===============",j,"=====================")
        print('La taille de X_train est:', X_train.shape)
        print('La taille de Y_train est:', Y_train.shape)
        print('La taille de X_test est:', X_test.shape)
        print('La taille de Y_test est:', Y_test.shape)'''

        # Créer le modèle KNN Regressor
        model = KNeighborsRegressor()

        # Définir les paramètres à tester dans la recherche par grille
        '''param_grid = {
            'n_neighbors': np.arange(1, 150),  # Increase the range up to 500
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Add 'algorithm' hyperparameter
            'leaf_size': np.arange(1, 60),  # Add 'leaf_size' hyperparameter
            'p': [1, 2],  # Add 'p' hyperparameter
            'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],  # Add 'chebyshev' in 'metric'
            'metric_params': [None],  # Add 'metric_params' hyperparameter
            'n_jobs': [None, -1]  # Add 'n_jobs' hyperparameter
        }'''
        param_grid = {
          'n_neighbors': np.arange(1, 100),
          'weights': ['uniform', 'distance'],
          'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
          }
        # Créer l'objet GridSearchCV
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

        # Effectuer la recherche par grille sur les données d'entraînement
        grid_search.fit(X_train, Y_train)

        print("\n***********************TOUR No ",j,"/10:************************")

        # Afficher les meilleurs paramètres trouvés
        print("Meilleurs paramètres trouvés :")
        print(grid_search.best_params_)

        '''#-------------Train---------------------
        # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
        y_pred_train = grid_search.predict(X_train)
        # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
        mse_train = mean_squared_error(Y_train, y_pred_train)
        rmse_train = np.sqrt(mse_train)
        print('Train:==========================================================')
        print('MSE : ',mse_train * 100,'%')
        print('RMSE : ',rmse_train * 100,'%')'''


        #-------------Test---------------------
        # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
        y_pred_test = grid_search.predict(X_test)
        # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
        mse_test = mean_squared_error(Y_test, y_pred_test)
        rmse_test = np.sqrt(mse_test)
        mae_test = mean_absolute_error(Y_test, y_pred_test)
        mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100

        print('\n=============================Test=============================')
        print('MSE : ',mse_test * 100,'%')
        print('RMSE : ',rmse_test * 100,'%')
        print('MAE : ',mae_test * 100,'%')
        print('MAPE : ',mape_test,'%')

        #score
        '''train_rmse_list.append(mse_train)'''
        mse_test_list.append(mse_test)
        rmse_test_list.append(rmse_test)
        mae_test_list.append(mae_test)
        mape_test_list.append(mape_test)
        
        
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print("time: " , (time.process_time()-t)/60,"min")
        print('')

    print("\n\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Total time: " , (time.process_time()-t)/60,"min")
                              
    print('All MSE: ',mse_test_list)
    print('\n\nmean MSE', np.mean(mse_test_list), " == ",np.mean(mse_test_list)*100,"%\n\n")
    
    print('All RMSE: ',rmse_test_list)
    print('\n\n******************************************************************************************')
    print('******************************************************************************************')
    print('***************** mean RMSE', np.mean(rmse_test_list), " ==> ",np.mean(rmse_test_list)*100,"% *****************")
    print('******************************************************************************************')
    print('******************************************************************************************\n\n')


    print('mean MAE : ', np.mean(mae_test_list) * 100,'%')
    print('mean MAPE : ', np.mean(mape_test_list),'%')

In [20]:
my_knn_regressor(train_fd001, test_fd001)


***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 44, 'weights': 'distance'}

=============================Test=============================
MSE :  2.5802291960564783 %
RMSE :  16.063091844525072 %
MAE :  12.169936862912797 %
MAPE :  30.24155216596081 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  16.539583333333333 min


***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 51, 'weights': 'distance'}

=============================Test=============================
MSE :  2.7529407227901586 %
RMSE :  16.591988195482056 %
MAE :  12.651848285923839 %
MAPE :  inf %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  32.776041666666664 min


***********************TOUR No  3 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 52, 'weights': 'dist

In [21]:
my_knn_regressor(train_fd002, test_fd002)


***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 46, 'weights': 'distance'}

=============================Test=============================
MSE :  1.4966849832743137 %
RMSE :  12.233907729234817 %
MAE :  9.632017416949143 %
MAPE :  31.839891272179372 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  73.8453125 min


***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 50, 'weights': 'distance'}

=============================Test=============================
MSE :  1.525722283550719 %
RMSE :  12.352013129651048 %
MAE :  9.742717467402255 %
MAPE :  29.209722566466635 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  147.62447916666667 min


***********************TOUR No  3 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 42, 'weights': 

In [22]:
my_knn_regressor(train_fd003, test_fd003)


***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 46, 'weights': 'distance'}

=============================Test=============================
MSE :  1.8424965960112967 %
RMSE :  13.57385942173889 %
MAE :  9.978296675556658 %
MAPE :  31.4214832073813 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  16.30625 min


***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 81, 'weights': 'distance'}

=============================Test=============================
MSE :  2.1131555113720992 %
RMSE :  14.536696706515203 %
MAE :  10.589937913769045 %
MAPE :  30.634746704016663 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  32.21067708333333 min


***********************TOUR No  3 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 64, 'weights': 'dis

In [23]:
my_knn_regressor(train_fd004, test_fd004)


***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 34, 'weights': 'distance'}

=============================Test=============================
MSE :  1.6714430639173192 %
RMSE :  12.92843015960298 %
MAE :  9.420596974886013 %
MAPE :  35.40657506508983 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  68.19296875 min


***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 36, 'weights': 'distance'}

=============================Test=============================
MSE :  1.7563598420910937 %
RMSE :  13.2527726989151 %
MAE :  9.739510507672053 %
MAPE :  33.394209478107236 %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  137.92864583333332 min


***********************TOUR No  3 /10:************************
Meilleurs paramètres trouvés :
{'metric': 'manhattan', 'n_neighbors': 39, 'weights': 'd